In [5]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import cv2
from myshow import myshow
from scipy import linalg
from scipy.io import loadmat, savemat
import sys

sys.path.append('include')
import cardiacDicomGlobals as cdg
import reconPipelineFunctions as recon
import validationPipelineFunctions as validate

%matplotlib inline


def closestIndex(f, faxis):
    return np.argmin(np.abs(faxis-f))

In [6]:
doGlobalFreqSearch = False
#0 good bicarb
#2: good pyruvate ex
#3: good pyruvate ex !!
#7 good bicarb

In [7]:
fmin = -45
fmax = 45
df = 5
nf = int((fmax - fmin)/df + 1)
freqAxis = np.linspace(fmin, fmax, nf)

gfmin = -15
gfmax = 15
ngf = int((gfmax - gfmin)/df + 1)
globalFreqSearch = np.linspace(gfmin, gfmax, ngf) 

globalOnResInd = cdg.closestIndex(0, globalFreqSearch)
onResInd = closestIndex(0, freqAxis)


In [8]:

pind = 0
for currentPatient in cdg.patientList:

    print('patient '+str(pind))

    cjmat = recon.generateMFICoefficients(freqAxis, currentPatient)

    
    
    b0mag = sitk.ReadImage(currentPatient.b0mag)
    b0phase = sitk.ReadImage(currentPatient.b0phase)
    loc = sitk.ReadImage(currentPatient.loc)
    c13 = sitk.ReadImage(currentPatient.c13)    
    maskSITK = recon.getHeartMask(currentPatient, loc)
    b0phasef, b0phasePixels, medianFiltered = recon.filterB0Map(b0phase)
    
    
    b0phase_r = recon.resampleSITKImage(c13,b0phasef)
    b0mag_r = recon.resampleSITKImage(c13,b0mag)
    mask_r = recon.resampleSITKImage(c13,maskSITK)
    loc_r = recon.resampleSITKImage(c13,loc)
    maskPixels = np.squeeze(sitk.GetArrayFromImage(mask_r))
    
    locMask = recon.getThresholdFromLoc(loc_r)

    
    
    matFileBase = currentPatient.mfr
    sampleFile= 'matFilesIntermediate/' + matFileBase + '_f' + str(1) + '.mat'
    array = loadmat(sampleFile)['bb']
    array = np.squeeze(np.cdouble(array))
    
    mc = currentPatient.multichannel
    mfr, mfrc = recon.loadMultiFrequencyFiles(currentPatient, freqAxis, False)
    mf, mfi, globalFreqSearch = recon.runMFI(globalFreqSearch, freqAxis, b0phase_r, locMask, mfr, mfrc, cjmat, mc)
    
    kernelSize = 5
    maskPixels = np.squeeze(sitk.GetArrayFromImage(mask_r))
    peakTime = validate.findPeakSignalImages(mfi, maskPixels, globalOnResInd)                          
    print('peak signal time indices: '+str(peakTime))

    peakSignalUncorr, peakSignalMFI, peakSignalSeg = validate.applyObjectiveMetric(freqAxis,globalFreqSearch, mfr, mfi, mf, peakTime, maskPixels)

    im = 0  # which met to optimize on  
    bestGlobalShiftUncorr = np.argmax(peakSignalUncorr[im,:])    
    bestGlobalShift       = np.argmax(peakSignalSeg[im,:])

    print('best global shift is at '+str(globalFreqSearch[bestGlobalShift])+' Hz')
    print('best global shift-only is at '+str(globalFreqSearch[bestGlobalShiftUncorr])+' Hz')

    
    # no correction
    img_unc = np.abs(np.squeeze(mfr[:, :, :, :, onResInd]))
    #global shift only 
    img_gs = np.abs(np.squeeze(mfr[:, :, :, :, closestIndex(globalFreqSearch[bestGlobalShift], freqAxis)]))
    #global shift + freq segmented
    img_seg = np.abs(np.squeeze(mf[:,:,:,:,bestGlobalShift]))
    #global shift + MFI
    img_mfi = np.abs(np.squeeze(mfi[:,:,:,:,bestGlobalShift]))

    
    
    # peak signals within circle wedgies
    numThetaBins = 10
    circleThickness = 15
    circlocs = currentPatient.circle
    it = int(peakTime[0])
    img_bicarb_unc = np.squeeze(img_unc[:,:,it,0])
    img_bicarb_gs = np.squeeze(img_gs[:,:,it,0])
    img_bicarb_mfi = np.squeeze(img_mfi[:,:,it,0])
    
    img_bicarb_unc = img_bicarb_unc / np.std(img_bicarb_unc[1:20,1:20])
    img_bicarb_gs  = img_bicarb_gs / np.std(img_bicarb_gs[1:20,1:20])
    img_bicarb_mfi = img_bicarb_mfi / np.std(img_bicarb_mfi[1:20,1:20])
    
    locPixels = np.squeeze(sitk.GetArrayFromImage(loc))
    binImages = recon.divideMyocardiumIntoBins(locPixels, circlocs, numThetaBins, circleThickness)
    binImages_res = recon.resampleCardiacBinImages(binImages, loc, c13)
    
    circ_profile_unc = np.zeros(numThetaBins)
    circ_profile_gs = np.zeros(numThetaBins)
    circ_profile_mfi = np.zeros(numThetaBins)
    for x in range(0, numThetaBins): 
        thisBin = binImages_res[x][:][:]
        pixels_unc = img_bicarb_unc[thisBin > 0]
        pixels_gs = img_bicarb_gs[thisBin > 0]
        pixels_mfi = img_bicarb_mfi[thisBin > 0]

        circ_profile_unc[x] = np.mean(np.quantile(pixels_unc, 0.75))
        circ_profile_gs[x] = np.mean(np.quantile(pixels_gs, 0.75))
        circ_profile_mfi[x] = np.mean(np.quantile(pixels_mfi, 0.75))
    
    # radial line profiles
    segRadius = 20
    NRadialSamples = 50
    numThetaBins = 10

    circleLocsC13Coords = recon.resampleCircleCenterCoords(circlocs, loc, c13)
    endPoints = recon.generateRadialEndpoints(circleLocsC13Coords, numThetaBins, segRadius)
    readout = loadmat(currentPatient.readout)
    
    
    zi_unc = recon.generateLineProfilesBinned(img_bicarb_unc, numThetaBins, NRadialSamples, circleLocsC13Coords, segRadius)
    zi_gs = recon.generateLineProfilesBinned(img_bicarb_gs, numThetaBins, NRadialSamples, circleLocsC13Coords, segRadius)
    zi_mfi = recon.generateLineProfilesBinned(img_bicarb_mfi, numThetaBins, NRadialSamples, circleLocsC13Coords, segRadius)

    
    
    
    scaleFactor = (segRadius / NRadialSamples) * (readout['fov'][0][0]/mfi.shape[0]) * 1000
    
    fwhm_unc = np.zeros(numThetaBins)
    fwhm_gs = np.zeros(numThetaBins)
    fwhm_mfi = np.zeros(numThetaBins)
    for i in range(numThetaBins):
        fwhm_unc[i] =  scaleFactor*recon.fwhm(zi_unc[i][:])
        fwhm_gs[i] =  scaleFactor*recon.fwhm(zi_gs[i][:])
        fwhm_mfi[i] =  scaleFactor*recon.fwhm(zi_mfi[i][:])
    
    
    dicomKey = 'ImagingFrequency'
    keys = loc.GetMetaDataKeys()
    h1freq = float(b0mag.GetMetaData('0018|0084'))
    c13FreqAdjustment = globalFreqSearch[bestGlobalShift] * 1e-6
    c13freq = float(c13.GetMetaData('0018|0084'))

    print('1H freq (dicom header) = '+ str(h1freq))
    print('13C freq (dicom header) = '+ str(c13freq))
    print('13C freq adjustment (from reconstruction) = '+ str(c13FreqAdjustment))

    ratio = (c13freq+c13FreqAdjustment)/h1freq
    print('ratio = '+str(ratio))

    readoutStr = 'long'
    if currentPatient.readout.find('short') > -1:
        readoutStr ='short'

    fileDict = dict()
    fileDict["img_unc"] = img_unc
    fileDict['img_gs'] = img_gs
    fileDict['img_seg'] = img_seg
    fileDict['img_mfi'] = img_mfi 
    fileDict['h1freq'] = h1freq
    fileDict['c13freq'] = c13freq
    fileDict['globalFreqSearch'] = globalFreqSearch
    fileDict['peakSignalUncorr'] = peakSignalUncorr
    fileDict['peakSignalSeg'] = peakSignalSeg
    fileDict['peakSignalMFI'] = peakSignalMFI
    fileDict['freqAxis'] = freqAxis
    fileDict['readoutStr'] = readoutStr
    fileDict['peakTime'] = peakTime

    
    
    fileDict['circ_profile_unc'] = circ_profile_unc
    fileDict['circ_profile_gs'] = circ_profile_gs
    fileDict['circ_profile_mfi'] = circ_profile_mfi
    fileDict['fwhm_unc'] = fwhm_unc
    fileDict['fwhm_gs'] = fwhm_gs
    fileDict['fwhm_mfi'] = fwhm_mfi
    

    outFileName = "B0CorrectedImages/"+currentPatient.mfr+".mat"
    savemat(outFileName, fileDict)

    pind +=1

patient 0
Progress:100.0%...image array shape
(128, 128, 8, 3, 19)
peak signal time indices: [3. 3. 1.]
best global shift is at 0.0 Hz
best global shift-only is at 10.0 Hz
Progress:100.0%...1H freq (dicom header) = 127.736282
13C freq (dicom header) = 32.12431
13C freq adjustment (from reconstruction) = 0.0
ratio = 0.2514893145238093
patient 1
Progress:100.0%...image array shape
(128, 128, 8, 3, 19)
peak signal time indices: [4. 2. 1.]
best global shift is at 15.0 Hz
best global shift-only is at 15.0 Hz
Progress:100.0%...1H freq (dicom header) = 127.736282
13C freq (dicom header) = 32.12431
13C freq adjustment (from reconstruction) = 1.4999999999999999e-05
ratio = 0.2514894319532488
patient 2
Progress:100.0%...image array shape
(128, 128, 8, 3, 19)
peak signal time indices: [2. 2. 1.]
best global shift is at 5.0 Hz
best global shift-only is at 5.0 Hz
Progress:100.0%...1H freq (dicom header) = 127.736239
13C freq (dicom header) = 32.124299
13C freq adjustment (from reconstruction) = 4.9